In [37]:
import wave
import scipy as sp
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from scipy.fftpack import fft
import scipy.signal as sg
from scipy import signal
import pandas as pd
import scipy.interpolate as itpl
import resampy
import math

#信号読み込み
#データベース生成
#測位

def chirp_exp(numSamples, chirpLen_s, start_Hz, stop_Hz, phase_rad):
    # チャープの生成関数,chirpaは生成したチャープ信号の配列を返す
    times_s = np.linspace(0, chirpLen_s, numSamples) # Chirp times.
    k = (stop_Hz - start_Hz) / chirpLen_s # Chirp rate.
    sweepFreqs_Hz = (start_Hz + k/2. * times_s) * times_s
    chirpa = np.array(np.exp(phase_rad*1j-2*np.pi*1j*sweepFreqs_Hz))
    return chirpa

def readwav(file1):
    wr = wave.open(file1, 'r')
    # waveファイルが持つ性質を取得
    ch = wr.getnchannels()
    width = wr.getsampwidth()
    fr = wr.getframerate()
    fn = wr.getnframes()
    data = wr.readframes(wr.getnframes())
    wr.close()
    X = np.frombuffer(data, dtype=np.int16)
    # 波全体の時間幅
    t = np.arange(0,len(X))/int(fr)
    # 1秒間の時間幅
    t_1sec = np.arange(0,int(fr))/int(fr)
    # 時間刻み幅
    dt = 1/int(fr)
    # 開始時間の指標
    start = int(2.0/dt)
    # 終了時間の指標
    end = int(5.0/dt)
    # plt.plot(t_1sec,X_1sec)
    return X


def create_db():
    sound_db = np.zeros(144*10*81*76).reshape(81,76,144*10)
    sound_db[:,:,:] = np.nan
    speaker_height = 100
    for h in range(0,4):
        mic_deg = -40
        for i in range(0,9):
            # wav読み込み
            sound_sample = readwav('vivesound_0322/s'+str(speaker_height)+'m'+str(mic_deg)+'.wav')
            start_hz = 1000
            for j in range(0,10):
                #チャープ信号の生成
                chirp = chirp_exp(144,0.003,start_hz,start_hz+1000,np.pi*0.5)
                # 相互相関
                corr = sg.correlate(sound_sample[50000:50000+48000*3],chirp,mode='same',method='auto')
                #最大値のインデックス見つける
                index = corr.argmax()
                #1つ分の波の抽出
                X_1sec = sound_sample[50000+index-72:50000+index+72]
                # FFT
                X_fft = np.abs(fft(X_1sec))
                # データベース構築
                sound_db[mic_deg+40,speaker_height-100,0+j*144:144+j*144] = X_fft
                start_hz += 1000
            mic_deg += 10
        speaker_height += 25

    #ここから秋間補間，マイク方位で補間した後スピーカ高さで補間
    Akima = np.zeros(144*10*81*76).reshape(81,76,144*10)
    Akima[:,:,:] = np.nan
    #マイク方位で補間
    count = 0
    for j in range(0,4):
        sound_db_d = pd.DataFrame(sound_db[:,count,:])
        sound_db_d.astype('float64')
        Akima[:,count,:] = sound_db_d.interpolate('akima')
        count += 25

    #スピーカ高さで補間
    for i in range(0,81):
        sound_db_d = pd.DataFrame(Akima[i,:,:])
        sound_db_d.astype('float64')
        Akima[i,:,:] = sound_db_d.interpolate('akima')
    
    #Akima = pd_Zrf_d.interpolate('akima')
    print('Akima done!!')
    return Akima
    
# 残差平方和
def rss(y, t):
    f = np.sum((y-t).T*(y-t))
    return f

def normalzie(x, amin=0, amax=1):
    xmax = x.max()
    xmin = x.min()
    if xmin == xmax:
        return np.ones_like(x)
    return (amax - amin) * (x - xmin) / (xmax - xmin) + amin
#normalized_x = normalzie(x, amin=-1, amax=1)
    
def estimate(db,file):
    #データベースを読み込んで、残差平方和を計測
    rss_db = np.zeros(81*76).reshape(81,76)
    file_db = np.zeros(144*10)
    #計測データを切り抜き
    start_hz = 1000
    for j in range(0,10):
        #チャープ信号の生成
        chirp = chirp_exp(144,0.003,start_hz,start_hz+1000,np.pi*0.5)
        # 相互相関
        corr = sg.correlate(file[50000:50000+48000*3], chirp, mode='same', method='auto')
        #最大値のインデックス見つける
        index = corr.argmax()
        #1つ分の波の抽出
        X_1sec = file[50000+index-72:50000+index+72]
        # FFT
        X_fft = np.abs(fft(X_1sec))
        # データベース構築
        file_db[j*144:144+j*144] = X_fft
        start_hz += 1000
    #print(file_db)
    #print(db)
    for j in range(0,81):
        for i in range(0,76):
            rss_ans = rss(normalzie(db[j,i],0,1),normalzie(file_db[:],0,1))
            rss_db[j,i] = rss_ans
    #print(rss_db)
    estimate_mic_speaker = np.unravel_index(np.argmin(rss_db), rss_db.shape)
    estimate_mic_speaker = list(estimate_mic_speaker)
    chirp = chirp_exp(144,0.003,1000,1100,np.pi*0.5)
    distance = (100.0*max(np.abs(sg.correlate(db[estimate_mic_speaker[0],estimate_mic_speaker[1],0:144],chirp, mode='same', method='auto'))))/max(np.abs(sg.correlate(file_db[0:144],chirp, mode='same', method='auto')))
    print('(推定マイクx座標,推定マイク高さ,推定距離)'+str(int(math.sin(estimate_mic_speaker[0])*distance))+'cm,'+str(int(estimate_mic_speaker[1]+100))+'cm,'+str(int(distance))+'cm') 

if __name__ == '__main__':
    #データベース作成
    db1 = create_db()
    #計測信号読み込み
    signal = readwav('vivesound_0322/test5.wav')
    # 最小二乗法によって測位
    estimate(db1,signal)

Akima done!!
(推定マイクx座標,推定マイク高さ,推定距離)-50cm,139cm,78cm


In [ ]:
(推定マイクx座標,推定マイク高さ,推定距離)-21cm,174cm,134cm,  38cm,168cm,126cm
(推定マイクx座標,推定マイク高さ,推定距離)-23cm,150cm,82cm, -34,158,158
(推定マイクx座標,推定マイク高さ,推定距離)-101cm,175cm,132cm, -75,146,160
(推定マイクx座標,推定マイク高さ,推定距離)-85cm,116cm,95cm, 3,133,119
(推定マイクx座標,推定マイク高さ,推定距離)-50cm,139cm,78cm,16,112,106
(推定マイクx座標,推定マイク高さ,推定距離)86cm,100cm,129cm,-29,114,132